# Chapter 1: Understanding Tokenization

Welcome to the first notebook in our LLM from Scratch series! In this chapter, we'll explore **tokenization** - the fundamental first step in processing text for language models.

## What You'll Learn

1. **What is tokenization** and why it's essential for LLMs
2. **Different tokenization approaches**: character-level, word-level, and subword-level
3. **Byte Pair Encoding (BPE)**: the algorithm used by GPT models
4. **Hands-on practice** with our Tokenizer implementation
5. **Special tokens** and their purposes
6. **Vocabulary size** and its trade-offs

Let's dive in!

## 1. What is Tokenization?

**Tokenization** is the process of converting raw text into a sequence of integers (token IDs) that a neural network can process.

### Why do we need it?

Neural networks can only process numerical data, not text directly. We need a systematic way to:
- Convert text → numbers (encoding)
- Convert numbers → text (decoding)
- Handle a fixed vocabulary efficiently
- Deal with rare or unknown words

Let's visualize this process:

```
Text:  "Hello, world!" 
   ↓ (tokenize)
Tokens: ["Hello", ",", " world", "!"]
   ↓ (encode)
IDs:   [15496, 11, 995, 0]
   ↓ (neural network)
Output IDs: [15496, 11, 995, 0, 1374, 389, 345, 30]
   ↓ (decode)
Text:  "Hello, world! How are you?"
```

## 2. Different Tokenization Approaches

There are three main approaches to tokenization:

### Character-Level Tokenization
- **Vocabulary**: Just 26 letters + punctuation (~100 tokens)
- **Pros**: Small vocabulary, can handle any word
- **Cons**: Very long sequences, loses word meaning
- **Example**: "Hello" → ["H", "e", "l", "l", "o"] → [7, 4, 11, 11, 14]

### Word-Level Tokenization
- **Vocabulary**: Every unique word (~100k-1M tokens)
- **Pros**: Preserves word meaning, shorter sequences
- **Cons**: Huge vocabulary, can't handle unknown words
- **Example**: "Hello world" → ["Hello", "world"] → [23415, 1002]

### Subword-Level Tokenization (BPE)
- **Vocabulary**: Frequently occurring subwords (~50k tokens)
- **Pros**: Balance between vocabulary size and sequence length
- **Cons**: Slightly more complex algorithm
- **Example**: "unhappiness" → ["un", "happiness"] → [403, 12084]

**Modern LLMs like GPT use subword tokenization** because it provides the best trade-off!

## 3. Byte Pair Encoding (BPE)

BPE is an iterative algorithm that builds a vocabulary of subword units:

### Algorithm Steps:

1. **Initialize**: Start with individual characters as tokens
2. **Count pairs**: Find the most frequent pair of adjacent tokens
3. **Merge**: Create a new token for this pair
4. **Repeat**: Go to step 2 until vocabulary reaches desired size

### Example:

```
Corpus: "low", "lower", "lowest", "newer", "wider"

Initial: ['l', 'o', 'w', 'e', 'r', 'n', 'i', 'd']

Iteration 1: Most frequent pair is ('e', 'r')
Merge → 'er'
Vocabulary: ['l', 'o', 'w', 'e', 'r', 'n', 'i', 'd', 'er']

Iteration 2: Most frequent pair is ('l', 'o')
Merge → 'lo'
Vocabulary: ['l', 'o', 'w', 'e', 'r', 'n', 'i', 'd', 'er', 'lo']

... continue until vocabulary size = 50,000
```

### Key Benefits:

- **Common words** become single tokens ("the" → [262])
- **Rare words** are split into frequent subwords ("unhappiness" → ["un", "happiness"])
- **Unknown words** can always be represented (worst case: fall back to bytes)
- **Efficient** vocabulary size (~50k vs. millions for word-level)

## 4. Hands-On: Using Our Tokenizer

Let's import and use the Tokenizer from our LLM implementation!

In [ ]:
# Import our tokenizer
import sys
sys.path.append('..')  # Add parent directory to path

from src.llm import Tokenizer

# Create a tokenizer instance (uses GPT-2's BPE encoding)
tokenizer = Tokenizer()

print(f"Tokenizer: {tokenizer}")
print(f"Vocabulary size: {tokenizer.vocab_size:,}")
print(f"End-of-sequence token ID: {tokenizer.eos_token_id}")

### 4.1 Basic Encoding and Decoding

In [ ]:
# Encode text to token IDs
text = "Hello, world!"
token_ids = tokenizer.encode(text)

print(f"Original text: {text!r}")
print(f"Token IDs: {token_ids}")
print(f"Number of tokens: {len(token_ids)}")

# Decode back to text
decoded_text = tokenizer.decode(token_ids)
print(f"\nDecoded text: {decoded_text!r}")
print(f"Round-trip successful: {text == decoded_text}")

### 4.2 Exploring How Words are Tokenized

Let's see how different words are split into tokens:

In [ ]:
# Helper function to visualize tokenization
def show_tokenization(text: str) -> None:
    """Show how text is tokenized into subwords."""
    token_ids = tokenizer.encode(text)
    
    # Decode each token individually to see the subwords
    tokens = [tokenizer.decode([tid]) for tid in token_ids]
    
    print(f"Text: {text!r}")
    print(f"Tokens ({len(tokens)}): {tokens}")
    print(f"Token IDs: {token_ids}")
    print()

# Try different examples
show_tokenization("Hello")                    # Common word
show_tokenization("unhappiness")              # Rare word with prefix
show_tokenization("ChatGPT")                   # Modern term
show_tokenization("supercalifragilisticexpialidocious")  # Very rare word
show_tokenization("123-456-7890")              # Numbers and punctuation
show_tokenization("こんにちは")                # Non-English (Japanese)

### Observations:

- **Common words** like "Hello" are often single tokens
- **Rare words** are broken into meaningful subwords
- **Unknown words** fall back to character or byte-level representation
- **Spaces** are included in the tokenization (note the leading space in some tokens)
- **Non-English** text uses byte-level encoding

### 4.3 Batch Processing

In [ ]:
# Encode multiple texts at once
texts = [
    "The quick brown fox jumps over the lazy dog.",
    "Machine learning is fascinating!",
    "LLMs can generate human-like text."
]

batch_ids = tokenizer.encode_batch(texts)

for i, (text, ids) in enumerate(zip(texts, batch_ids), 1):
    print(f"Text {i}: {text}")
    print(f"Tokens: {len(ids)}")
    print(f"IDs: {ids}")
    print()

# Decode batch
decoded_texts = tokenizer.decode_batch(batch_ids)
print(f"Decoding successful: {texts == decoded_texts}")

## 5. Special Tokens

Tokenizers use **special tokens** to mark important positions in sequences:

- **BOS (Beginning of Sequence)**: Marks the start of a text
- **EOS (End of Sequence)**: Marks the end of a text  
- **PAD (Padding)**: Fills sequences to the same length
- **UNK (Unknown)**: Represents unknown tokens (rare in BPE)

Note: GPT-2 uses `<|endoftext|>` for both BOS and EOS.

In [ ]:
# Special token IDs
print(f"BOS token ID: {tokenizer.bos_token_id}")
print(f"EOS token ID: {tokenizer.eos_token_id}")

# Encoding with special tokens
text = "Hello, world!"
ids_no_special = tokenizer.encode(text, add_special_tokens=False)
ids_with_special = tokenizer.encode(text, add_special_tokens=True)

print(f"\nWithout BOS: {ids_no_special}")
print(f"With BOS:    {ids_with_special}")
print(f"BOS added at position 0: {ids_with_special[0] == tokenizer.bos_token_id}")

## 6. Vocabulary Size Trade-offs

The choice of vocabulary size has important implications:

### Small Vocabulary (e.g., 1,000 tokens)
- ✅ **Pros**: Smaller embedding matrix, less memory
- ❌ **Cons**: Longer sequences, loses semantic meaning

### Large Vocabulary (e.g., 1,000,000 tokens)
- ✅ **Pros**: Shorter sequences, better word representation
- ❌ **Cons**: Huge embedding matrix, rare tokens undertrained

### GPT-2's Choice: 50,257 tokens
- ⚖️ **Sweet spot** balancing all considerations
- Most common words and subwords are single tokens
- Rare words split into 2-3 tokens typically
- Embeddings are manageable (~200MB for 768 dimensions)

In [ ]:
# Let's see how sequence length varies with different texts
def compare_lengths(texts: list[str]) -> None:
    """Compare character length vs. token length."""
    print(f"{'Text':<50} {'Chars':<8} {'Tokens':<8} {'Ratio':<8}")
    print("-" * 80)
    
    for text in texts:
        tokens = tokenizer.encode(text)
        ratio = len(text) / len(tokens) if tokens else 0
        print(f"{text:<50} {len(text):<8} {len(tokens):<8} {ratio:<8.2f}")

test_texts = [
    "Hello",
    "The quick brown fox",
    "I love machine learning and neural networks!",
    "Tokenization is fundamental for NLP.",
    "supercalifragilisticexpialidocious",
]

compare_lengths(test_texts)

### Observations:

- On average, **1 token ≈ 4 characters** in English
- Common words have better compression ratios
- Rare/long words have worse compression ratios

## 7. Practical Exercise

Try tokenizing your own text! Experiment with:
- Different languages
- Code snippets
- Special characters and emojis
- Very long words

In [ ]:
# Your turn! Try different texts
my_text = "Write your own text here!"

show_tokenization(my_text)

## 8. Key Takeaways

Let's recap what we've learned:

1. **Tokenization converts text → numbers** so neural networks can process it

2. **BPE (Byte Pair Encoding)** is the algorithm used by modern LLMs:
   - Iteratively merges frequent character pairs
   - Creates a vocabulary of ~50k subword tokens
   - Balances vocabulary size and sequence length

3. **Common words** become single tokens, **rare words** split into subwords

4. **Special tokens** (BOS/EOS/PAD) help mark sequence boundaries

5. **Vocabulary size** (~50k) is carefully chosen to balance:
   - Model size (embedding matrix)
   - Sequence length
   - Semantic representation

6. **Our tokenizer** uses the same BPE encoding as GPT-2/GPT-3 (tiktoken)

## Next Steps

Now that you understand how text is converted to tokens, we're ready to explore the **attention mechanism** - the core innovation that makes transformers so powerful!

Continue to **Notebook 02: Attention Mechanism** →

---

## Further Reading

- [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/abs/1508.07909) (Original BPE paper)
- [tiktoken documentation](https://github.com/openai/tiktoken)
- [Hugging Face Tokenizers](https://huggingface.co/docs/tokenizers/index)